# Adapting NJ's Tensor code:

In [120]:
import numpy as np
import os
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
###  Models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import make_pipeline
### Classes
from src.DataLoader import DataLoader
from src.Tensor import *


In [19]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# TEST functions 

In [131]:
ACC_datas = DataLoader().load_ACC_data( "S1","morning")

In [132]:
ACC_datas

,X_acceleration,Y_acceleration,Z_acceleration,UNIX_time
0,-62.0,18.0,-2.0,1.675144e+09
1,-63.0,18.0,-2.0,1.675144e+09
2,-61.0,15.0,-4.0,1.675144e+09
3,-62.0,15.0,-8.0,1.675144e+09
4,-64.0,14.0,-9.0,1.675144e+09
...,...,...,...,...
57595,-56.0,20.0,29.0,1.675146e+09
57596,-53.0,19.0,30.0,1.675146e+09
57597,-58.0,22.0,33.0,1.675146e+09
57598,-56.0,20.0,31.0,1.675146e+09


In [133]:
# # NJ's code :
# dfs = []

# for walk in range(0,46):
#         df = pd.read_csv(f"data2/{walk}.csv", names = ["x",
#         "y", "z"], dtype=float)
#                 # Load the time when the data measurement started from the first
#                 # value in the first row of the csv file
#         starting_time = df.iloc[0, 0]
#                 # Load the data sampling rate from the first value in the second
#                 # row of the csv file
#         sampling_rate = df.iloc[1, 0]
#                 # Removes the first two rows of the DataFrame, as they contain
#                 # info about the measurment starting time and sampling rate
#         #df = df.tail(-2).reset_index(drop=True)
#                 # Determine the number of samples in the data
#         number_of_samples = len(df)
#                 # Generate the time vector based on the measurement starting time
#                 # and sampling rate
#         time_vector = np.arange(starting_time, starting_time +
#             number_of_samples / sampling_rate, 1 / sampling_rate)
#                 #Add the UNIX time vector to the Pandas DataFrame
#         df["UNIX_time"] = time_vector
#                 # Calculate the total duration of the DataFrame (in seconds)
#         total_duration = df['UNIX_time'].max() - df['UNIX_time'].min()
#                 # Calculate the start time for the 30-minute window (center of data)
#         half_duration = total_duration / 2
#         center_time = df['UNIX_time'].min() + half_duration
#                 # Define the 30-minute window
#         window_start = center_time - 900  # 900 seconds = 30 minutes
#         window_end = center_time + 900
#                 # Extract data within a 30-minute window located at the center of
#                 # original DataFrame
#         df = df[(df['UNIX_time'] >= window_start) & (df['UNIX_time'] <= window_end)]
#                 # Return the desired data as a Pandas DataFrame
#         df = df.drop(columns='UNIX_time')
#         df = df.reset_index(drop=True)
#         dfs.append(df)

In [134]:
# # My code :

# dfs = []

# for walk in range(1,24):
#         df = pd.read_csv(f"../data/subject_{walk}/1.morning/ACC.csv", names = ["x",
#         "y", "z"], dtype=float)
#                 # Load the time when the data measurement started from the first
#                 # value in the first row of the csv file
#         starting_time = df.iloc[0, 0]
#                 # Load the data sampling rate from the first value in the second
#                 # row of the csv file
#         sampling_rate = df.iloc[1, 0]
#                 # Removes the first two rows of the DataFrame, as they contain
#                 # info about the measurment starting time and sampling rate
#         df = df.tail(-2).reset_index(drop=True)
#                 # Determine the number of samples in the data
#         number_of_samples = len(df)
#                 # Generate the time vector based on the measurement starting time
#                 # and sampling rate
#         time_vector = np.arange(starting_time, starting_time +
#             number_of_samples / sampling_rate, 1 / sampling_rate)
#                 #Add the UNIX time vector to the Pandas DataFrame
#         df["UNIX_time"] = time_vector
#                 # Calculate the total duration of the DataFrame (in seconds)
#         total_duration = df['UNIX_time'].max() - df['UNIX_time'].min()
#                 # Calculate the start time for the 30-minute window (center of data)
#         half_duration = total_duration / 2
#         center_time = df['UNIX_time'].min() + half_duration
#                 # Define the 30-minute window
#         window_start = center_time - 900  # 900 seconds = 30 minutes
#         window_end = center_time + 900
#                 # Extract data within a 30-minute window located at the center of
#                 # original DataFrame
#         df = df[(df['UNIX_time'] >= window_start) & (df['UNIX_time'] <= window_end)]
#                 # Return the desired data as a Pandas DataFrame
#         df = df.drop(columns='UNIX_time')
#         df = df.reset_index(drop=True)
#         dfs.append(df)

In [135]:
Events = {"morning": "1.morning", "evening": "2.evening"}  ### has to use self (DataLoader)
for events in Events:
    print(Events[events][2:])

morning
evening


In [136]:
# adapting code : using ACC_datas = DataLoader().load_ACC_data( "S1","morning")

dfs = []
Subject_list = [f'S{i}' for i in range(1,24)]

Subject_list
## has to use self (DataLoader)


['S1',
 'S2',
 'S3',
 'S4',
 'S5',
 'S6',
 'S7',
 'S8',
 'S9',
 'S10',
 'S11',
 'S12',
 'S13',
 'S14',
 'S15',
 'S16',
 'S17',
 'S18',
 'S19',
 'S20',
 'S21',
 'S22',
 'S23']

In [137]:
for subject in Subject_list:
    for events in Events:
        df = DataLoader().load_ACC_data(subject,Events[events][2:])
                # Load the time when the data measurement started from the first
                # value in the first row of the csv file
        starting_time = 1675144125
                # Load the data sampling rate from the first value in the second
                # row of the csv file
        sampling_rate = 32
                # Removes the first two rows of the DataFrame, as they contain
                # info about the measurment starting time and sampling rate
        # df = df.tail(-2).reset_index(drop=True)
                # Determine the number of samples in the data
        number_of_samples = len(df)
                # Generate the time vector based on the measurement starting time
                # and sampling rate
        time_vector = np.arange(starting_time, starting_time +
            number_of_samples / sampling_rate, 1 / sampling_rate)
                #Add the UNIX time vector to the Pandas DataFrame
        df["UNIX_time"] = time_vector
                # Calculate the total duration of the DataFrame (in seconds)
        total_duration = df['UNIX_time'].max() - df['UNIX_time'].min()
                # Calculate the start time for the 30-minute window (center of data)
        half_duration = total_duration / 2
        center_time = df['UNIX_time'].min() + half_duration
                # Define the 30-minute window
        window_start = center_time - 900  # 900 seconds = 30 minutes
        window_end = center_time + 900
                # Extract data within a 30-minute window located at the center of
                # original DataFrame
        df = df[(df['UNIX_time'] >= window_start) & (df['UNIX_time'] <= window_end)]
                # Return the desired data as a Pandas DataFrame
        df = df.drop(columns='UNIX_time')
        df = df.reset_index(drop=True)
        dfs.append(df)

In [138]:
dfs

[       X_acceleration  Y_acceleration  Z_acceleration
 0               -62.0            18.0            -2.0
 1               -63.0            18.0            -2.0
 2               -61.0            15.0            -4.0
 3               -62.0            15.0            -8.0
 4               -64.0            14.0            -9.0
 ...               ...             ...             ...
 57595           -56.0            20.0            29.0
 57596           -53.0            19.0            30.0
 57597           -58.0            22.0            33.0
 57598           -56.0            20.0            31.0
 57599           -58.0            19.0            33.0
 
 [57600 rows x 3 columns],
        X_acceleration  Y_acceleration  Z_acceleration
 0               -63.0           -15.0            25.0
 1               -65.0           -12.0            21.0
 2               -65.0           -10.0            19.0
 3               -65.0            -8.0            18.0
 4               -64.0            -8

In [139]:
len(dfs)

46

In [140]:
dfs[0].shape

(57600, 3)

In [141]:
dfs[0].dtypes

X_acceleration    float64
Y_acceleration    float64
Z_acceleration    float64
dtype: object

In [142]:
dfs[0].columns.to_list()

['X_acceleration', 'Y_acceleration', 'Z_acceleration']

In [143]:
# X_tensor = np.array([df[['X_acceleration', 'Y_acceleration', 'Z_acceleration']].values for df in dfs])

In [144]:
#### with columns names :
X_tensor = np.array([df[dfs[0].columns.to_list()].values for df in dfs])

In [145]:
X_tensor

array([[[-62.,  18.,  -2.],
        [-63.,  18.,  -2.],
        [-61.,  15.,  -4.],
        ...,
        [-58.,  22.,  33.],
        [-56.,  20.,  31.],
        [-58.,  19.,  33.]],

       [[-63., -15.,  25.],
        [-65., -12.,  21.],
        [-65., -10.,  19.],
        ...,
        [-66.,  -4.,  -2.],
        [-66.,  -4.,  -3.],
        [-66.,  -4.,  -3.]],

       [[-28., -20.,  56.],
        [-28., -20.,  56.],
        [-29., -20.,  55.],
        ...,
        [-39., -52.,   2.],
        [-43., -50.,   3.],
        [-43., -49.,   8.]],

       ...,

       [[-19.,  -1.,  62.],
        [-20.,   0.,  61.],
        [-18.,  -1.,  63.],
        ...,
        [-38., -54.,   7.],
        [-38., -53.,   9.],
        [-36., -53.,  11.]],

       [[-35., -54.,  -6.],
        [-35., -55.,  -7.],
        [-35., -54.,  -6.],
        ...,
        [-43., -17.,  48.],
        [-43., -17.,  48.],
        [-43., -17.,  48.]],

       [[-47.,  15.,  43.],
        [-47.,  14.,  42.],
        [-50.,  

In [146]:
X_tensor.shape

(46, 57600, 3)

In [147]:
sleepy_list = [11, 18, 0, 12, 10, 11, 5, 11, 2, 5, 9, 9, 17, 4, 2, 12, 12, 19, 13, 7, 11, 3, 5, 14, 14, 14, 12, 13, 11, 1, 11, 13, 11, 8, 19, 21, 22, 16, 15, 12, 26, 4, 19, 21, 18, 20]

sleepy_df = pd.DataFrame(sleepy_list)

sleepy_df["fatigue"] = sleepy_df[0]

sleepy_df = sleepy_df.drop(columns=0)

sleepy_df["is_tired"] = 0

sleepy_df['is_tired'] = (sleepy_df['fatigue'] > 11).astype(int)

In [148]:
sleepy_df

,fatigue,is_tired
0,11,0
1,18,1
2,0,0
3,12,1
4,10,0
5,11,0
6,5,0
7,11,0
8,2,0
9,5,0


In [149]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X_tensor, sleepy_df_6split["is_tired"], test_size=0.3)
# display(X_train.shape)
# display(X_test.shape)
# display(y_train.shape)
# display(y_test.shape)

In [150]:
# #repeats each line of the y_targets table 6 times
# sleepy_df_6split = pd.DataFrame(np.repeat(sleepy_df.values, 6, axis=0))
# sleepy_df_6split.columns = sleepy_df.columns
# sleepy_df_6split


In [151]:
# #after df normalisation, where the list of dataframes is generated, before it the tensor is created


# df_list = []
# #divides each df into 6ths
# for chunk in range(6):
#     #adds first 6th to the output list
#     df_c = df[:9600]
#     df_list.append(df_c)
#     #discards first 6th
#     df = df.iloc[9600:]

In [152]:
df

,X_acceleration,Y_acceleration,Z_acceleration
0,-47.0,15.0,43.0
1,-47.0,14.0,42.0
2,-50.0,14.0,40.0
3,-50.0,14.0,44.0
4,-52.0,14.0,42.0
...,...,...,...
57595,-23.0,-59.0,13.0
57596,-22.0,-59.0,13.0
57597,-24.0,-59.0,12.0
57598,-25.0,-59.0,12.0


In [153]:
def normalize_data(data_subject):
    num_transformer=make_pipeline(StandardScaler())
    num_transformer.fit(data_subject)
    return pd.DataFrame(num_transformer.transform(data_subject), columns=data_subject.columns)

In [154]:
dfs[0]

,X_acceleration,Y_acceleration,Z_acceleration
0,-62.0,18.0,-2.0
1,-63.0,18.0,-2.0
2,-61.0,15.0,-4.0
3,-62.0,15.0,-8.0
4,-64.0,14.0,-9.0
...,...,...,...
57595,-56.0,20.0,29.0
57596,-53.0,19.0,30.0
57597,-58.0,22.0,33.0
57598,-56.0,20.0,31.0


In [155]:
df1_normalize = normalize_data(dfs[0])

In [156]:
df1_normalize

,X_acceleration,Y_acceleration,Z_acceleration
0,-0.589457,-0.114222,-0.212423
1,-0.671152,-0.114222,-0.212423
2,-0.507761,-0.269832,-0.308387
3,-0.589457,-0.269832,-0.500315
4,-0.752848,-0.321702,-0.548297
...,...,...,...
57595,-0.099284,-0.010482,1.275020
57596,0.145802,-0.062352,1.323002
57597,-0.262675,0.093258,1.466948
57598,-0.099284,-0.010482,1.370984


In [157]:
df_copy = dfs.copy()
df_copy


[       X_acceleration  Y_acceleration  Z_acceleration
 0               -62.0            18.0            -2.0
 1               -63.0            18.0            -2.0
 2               -61.0            15.0            -4.0
 3               -62.0            15.0            -8.0
 4               -64.0            14.0            -9.0
 ...               ...             ...             ...
 57595           -56.0            20.0            29.0
 57596           -53.0            19.0            30.0
 57597           -58.0            22.0            33.0
 57598           -56.0            20.0            31.0
 57599           -58.0            19.0            33.0
 
 [57600 rows x 3 columns],
        X_acceleration  Y_acceleration  Z_acceleration
 0               -63.0           -15.0            25.0
 1               -65.0           -12.0            21.0
 2               -65.0           -10.0            19.0
 3               -65.0            -8.0            18.0
 4               -64.0            -8

In [158]:
len(df_copy)
# df_copy[0].shape

46

In [159]:
for i in range(0,len(df_copy)):
    df_copy[i] = normalize_data(df_copy[i])




# for data_subject in df_copy:
#     df_copy = normalize_data(data_subject)
# data_subject
df_copy

[       X_acceleration  Y_acceleration  Z_acceleration
 0           -0.589457       -0.114222       -0.212423
 1           -0.671152       -0.114222       -0.212423
 2           -0.507761       -0.269832       -0.308387
 3           -0.589457       -0.269832       -0.500315
 4           -0.752848       -0.321702       -0.548297
 ...               ...             ...             ...
 57595       -0.099284       -0.010482        1.275020
 57596        0.145802       -0.062352        1.323002
 57597       -0.262675        0.093258        1.466948
 57598       -0.099284       -0.010482        1.370984
 57599       -0.262675       -0.062352        1.466948
 
 [57600 rows x 3 columns],
        X_acceleration  Y_acceleration  Z_acceleration
 0           -0.743233       -0.771796        0.160711
 1           -0.874754       -0.629046        0.004586
 2           -0.874754       -0.533879       -0.073476
 3           -0.874754       -0.438712       -0.112508
 4           -0.808993       -0.4387

In [160]:
X_tensor_normalized =

SyntaxError: invalid syntax (735643725.py, line 1)

In [ ]:
X_tensor.shape

(46, 57600, 3)

# Models

## X_datas

In [122]:
datas_subjects_list = Tensor_data()
datas_subjects_list

[       X_acceleration  Y_acceleration  Z_acceleration
 0               -62.0            18.0            -2.0
 1               -63.0            18.0            -2.0
 2               -61.0            15.0            -4.0
 3               -62.0            15.0            -8.0
 4               -64.0            14.0            -9.0
 ...               ...             ...             ...
 57595           -56.0            20.0            29.0
 57596           -53.0            19.0            30.0
 57597           -58.0            22.0            33.0
 57598           -56.0            20.0            31.0
 57599           -58.0            19.0            33.0
 
 [57600 rows x 3 columns],
        X_acceleration  Y_acceleration  Z_acceleration
 0               -63.0           -15.0            25.0
 1               -65.0           -12.0            21.0
 2               -65.0           -10.0            19.0
 3               -65.0            -8.0            18.0
 4               -64.0            -8

In [126]:
display(len(datas_subjects_list))
display(datas_subjects_list[0].shape)

46

(57600, 3)

In [127]:
# normalized datas:
Xlist_normalized = normalize_list_data(datas_subjects_list)

In [128]:
Xlist_normalized

[       X_acceleration  Y_acceleration  Z_acceleration
 0           -0.589457       -0.114222       -0.212423
 1           -0.671152       -0.114222       -0.212423
 2           -0.507761       -0.269832       -0.308387
 3           -0.589457       -0.269832       -0.500315
 4           -0.752848       -0.321702       -0.548297
 ...               ...             ...             ...
 57595       -0.099284       -0.010482        1.275020
 57596        0.145802       -0.062352        1.323002
 57597       -0.262675        0.093258        1.466948
 57598       -0.099284       -0.010482        1.370984
 57599       -0.262675       -0.062352        1.466948
 
 [57600 rows x 3 columns],
        X_acceleration  Y_acceleration  Z_acceleration
 0           -0.743233       -0.771796        0.160711
 1           -0.874754       -0.629046        0.004586
 2           -0.874754       -0.533879       -0.073476
 3           -0.874754       -0.438712       -0.112508
 4           -0.808993       -0.4387

In [ ]:
X_tensor_normalized =

## Y_datas

In [130]:
Y_label = label_creator()
Y_label

,fatigue,is_tired
0,11,0
1,18,1
2,0,0
3,12,1
4,10,0
5,11,0
6,5,0
7,11,0
8,2,0
9,5,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, sleepy_df_6split["is_tired"], test_size=0.3)